$\color{red}{\text{REMARK:}}$  
This notebook makes use of multiprocessing.  
  
Since Jupyter Notebooks cannot properly cope with multiprocessing, this notebook may crash while computing the forecasts when using too much data.  
  
Therefore, this notebook is intended for development and debugging. When computing forecasts for all items, lease use multithreading/05_forecast_fbprophet_item_level.py.

# Import Packages

In [1]:
import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fbprophet import Prophet
from forecaster import run_prophet
from tqdm.notebook import tqdm
import time
from multiprocessing import Pool, cpu_count

# Forecast using Prophet on Item Level

In [2]:
# Import data
submission = pd.read_csv(
    f'{utils.get_m5_root_dir()}/data/input/sample_submission.csv')
df_calendar, df_sales, df_prices = utils.import_m5_data(reduce_memory=False)

In [3]:
# Method has been copied to seperate forecaster.py file otherwise it gets stuck in an infinitive loop.
# Jupyter notebooks cannot properly cope with multiprocessing

"""
def run_prophet(df_prophet):
    model = Prophet(daily_seasonality=True, yearly_seasonality=True)
    model.fit(df_prophet)
    future = model.make_future_dataframe(periods=28, include_history=False)
    forecast = model.predict(future)['yhat'].to_list()
    return forecast

"""

"\ndef run_prophet(df_prophet):\n    model = Prophet(daily_seasonality=True, yearly_seasonality=True)\n    model.fit(df_prophet)\n    future = model.make_future_dataframe(periods=28, include_history=False)\n    forecast = model.predict(future)['yhat'].to_list()\n    return forecast\n\n"

In [4]:
# brigns a pd.Series to a DataFrame of prophet's required structure


def create_prophet_df(pd_series):
    d_string = [f'd_{di}' for di in list(range(1, 1914))]
    ds = pd.date_range(start='2011-01-29', end='2016-04-24')
    df = pd.DataFrame({'ds': ds, 'y': pd_series[d_string].values})
    return df

In [5]:
# Recude size of data --- only for debugging
df_sales = df_sales.head(100)
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,HOBBIES_1_099_CA_1_validation,HOBBIES_1_099,HOBBIES_1,HOBBIES,CA_1,CA,2,1,1,1,...,0,1,0,1,2,0,0,0,0,2
96,HOBBIES_1_100_CA_1_validation,HOBBIES_1_100,HOBBIES_1,HOBBIES,CA_1,CA,1,2,2,1,...,2,3,1,3,0,0,1,1,1,1
97,HOBBIES_1_102_CA_1_validation,HOBBIES_1_102,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
98,HOBBIES_1_103_CA_1_validation,HOBBIES_1_103,HOBBIES_1,HOBBIES,CA_1,CA,25,8,5,22,...,7,8,4,0,10,2,4,0,2,5


In [6]:
# Iteratively compute forecasts on item level.

if __name__ == '__main__':

    start = time.time()

    # create a list of dataframes containing the sales figures of one item
    list_of_df_sales = [
        create_prophet_df(row) for i, row in df_sales.iterrows()
    ]

    # initiate a pool of workers
    pool = Pool(cpu_count())

    # paralell computation of forecats
    # list of lists containing the forecasts
    results = pool.map(run_prophet, tqdm(list_of_df_sales))

    # tranform list of lists to a dataframe
    results_df = pd.DataFrame(results,
                              columns=[f'F{di}' for di in list(range(1, 29))])

    # add columns with item's ids
    results_df.insert(0, 'id', df_sales['id'])

    # print computation stats
    print(
        f'Computation took {np.round((time.time() - start)/60,2)} minutes in total.'
    )
    print(
        f'Computation took {np.round((time.time()-start)/df_sales.shape[0],2)} seconds per item.'
    )


Computation took 0.98 minutes in total.
Computation took 0.59 seconds per item.


In [7]:
results_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.765524,0.825636,0.709930,0.700229,0.752582,0.899714,0.776378,0.724902,0.783688,...,0.655437,0.802138,0.679068,0.628654,0.689385,0.575150,0.567878,0.623709,0.775442,0.657890
1,HOBBIES_1_002_CA_1_validation,0.148058,0.159780,0.177024,0.228968,0.256769,0.347268,0.289613,0.236834,0.256940,...,0.430566,0.507257,0.432560,0.359923,0.357854,0.357690,0.389008,0.393453,0.458433,0.373722
2,HOBBIES_1_003_CA_1_validation,0.646831,0.656270,0.654694,0.667836,0.706775,0.782454,0.694020,0.641173,0.652129,...,0.733811,0.814480,0.730675,0.681940,0.696344,0.699444,0.716805,0.759348,0.837879,0.751429
3,HOBBIES_1_004_CA_1_validation,1.567754,1.245377,1.184262,1.593367,1.590055,2.680505,3.289942,1.549145,1.237448,...,1.690509,2.791603,3.408682,1.672184,1.361176,1.308076,1.721464,1.718398,2.804839,3.405891
4,HOBBIES_1_005_CA_1_validation,0.682394,0.602300,0.701623,0.633248,0.705551,1.146410,1.055758,0.533515,0.468903,...,0.849026,1.358161,1.335692,0.880029,0.878905,1.057580,1.067268,1.214675,1.726050,1.699779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,HOBBIES_1_099_CA_1_validation,1.190379,0.955930,0.922519,0.870352,1.206021,1.407599,1.434734,1.126603,0.889804,...,1.100097,1.312652,1.353159,1.060754,0.841924,0.826677,0.795173,1.153885,1.380692,1.434980
96,HOBBIES_1_100_CA_1_validation,1.010122,0.847250,0.764837,0.821225,0.986829,1.194994,1.043817,1.062881,0.893557,...,0.950123,1.139952,0.971611,0.975168,0.792436,0.691142,0.730169,0.880426,1.075699,0.914458
97,HOBBIES_1_102_CA_1_validation,0.100639,0.080290,0.089658,0.128725,0.109550,0.171142,0.159947,0.120135,0.101922,...,0.153226,0.212230,0.197578,0.153496,0.130278,0.135793,0.170077,0.145268,0.200472,0.182254
98,HOBBIES_1_103_CA_1_validation,2.621551,2.566551,3.107020,2.027167,2.451985,3.223404,3.299337,2.207273,2.125885,...,1.603293,2.417369,2.548675,1.524201,1.521865,2.130245,1.133057,1.654510,2.535466,2.732508


In [8]:
# Bring data to Kaggel's required form (evaluation & validation)
df_submission = results_df.append(
    results_df.replace(to_replace='validation', value='evaluation',
                       regex=True))
filename = utils.get_m5_root_dir(
) + '/data/submissions/fbprophet_on_item_level.csv'
#df_submission.to_csv(filename, index=False)
df_submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.765524,0.825636,0.709930,0.700229,0.752582,0.899714,0.776378,0.724902,0.783688,...,0.655437,0.802138,0.679068,0.628654,0.689385,0.575150,0.567878,0.623709,0.775442,0.657890
1,HOBBIES_1_002_CA_1_validation,0.148058,0.159780,0.177024,0.228968,0.256769,0.347268,0.289613,0.236834,0.256940,...,0.430566,0.507257,0.432560,0.359923,0.357854,0.357690,0.389008,0.393453,0.458433,0.373722
2,HOBBIES_1_003_CA_1_validation,0.646831,0.656270,0.654694,0.667836,0.706775,0.782454,0.694020,0.641173,0.652129,...,0.733811,0.814480,0.730675,0.681940,0.696344,0.699444,0.716805,0.759348,0.837879,0.751429
3,HOBBIES_1_004_CA_1_validation,1.567754,1.245377,1.184262,1.593367,1.590055,2.680505,3.289942,1.549145,1.237448,...,1.690509,2.791603,3.408682,1.672184,1.361176,1.308076,1.721464,1.718398,2.804839,3.405891
4,HOBBIES_1_005_CA_1_validation,0.682394,0.602300,0.701623,0.633248,0.705551,1.146410,1.055758,0.533515,0.468903,...,0.849026,1.358161,1.335692,0.880029,0.878905,1.057580,1.067268,1.214675,1.726050,1.699779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,HOBBIES_1_099_CA_1_evaluation,1.190379,0.955930,0.922519,0.870352,1.206021,1.407599,1.434734,1.126603,0.889804,...,1.100097,1.312652,1.353159,1.060754,0.841924,0.826677,0.795173,1.153885,1.380692,1.434980
96,HOBBIES_1_100_CA_1_evaluation,1.010122,0.847250,0.764837,0.821225,0.986829,1.194994,1.043817,1.062881,0.893557,...,0.950123,1.139952,0.971611,0.975168,0.792436,0.691142,0.730169,0.880426,1.075699,0.914458
97,HOBBIES_1_102_CA_1_evaluation,0.100639,0.080290,0.089658,0.128725,0.109550,0.171142,0.159947,0.120135,0.101922,...,0.153226,0.212230,0.197578,0.153496,0.130278,0.135793,0.170077,0.145268,0.200472,0.182254
98,HOBBIES_1_103_CA_1_evaluation,2.621551,2.566551,3.107020,2.027167,2.451985,3.223404,3.299337,2.207273,2.125885,...,1.603293,2.417369,2.548675,1.524201,1.521865,2.130245,1.133057,1.654510,2.535466,2.732508


In [9]:
# Submit via Kaggle API or with website https://www.kaggle.com/c/m5-forecasting-accuracy/submissions
# !kaggle competitions submit -c m5-forecasting-accuracy -f data/submissions/fbprophet_on_item_level.csv -m"fbprophet_on_item_level.csv"